pip install optuna

In [2]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 348 kB 6.9 MB/s 
     |████████████████████████████████| 81 kB 7.5 MB/s 
     |████████████████████████████████| 209 kB 48.9 MB/s 
     |████████████████████████████████| 78 kB 7.2 MB/s 
     |████████████████████████████████| 147 kB 50.3 MB/s 
     |████████████████████████████████| 49 kB 6.2 MB/s 
     |████████████████████████████████| 112 kB 41.5 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=5e7e757d65310d0c3b9489e1685a28e51b0d851223b945cf9118f93b3ffbb5f2
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


import optuna

!pip install xgboost

In [3]:
!pip install xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import optuna
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [7]:
df = pd.read_csv('Admission_Prediction.csv')

In [8]:
df

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337.0,118.0,4.0,4.5,4.5,9.65,1,0.92
1,2,324.0,107.0,4.0,4.0,4.5,8.87,1,0.76
2,3,NaN,104.0,3.0,3.0,3.5,8.00,1,0.72
3,4,322.0,110.0,3.0,3.5,2.5,8.67,1,0.80
4,5,314.0,103.0,2.0,2.0,3.0,8.21,0,0.65
...,...,...,...,...,...,...,...,...,...
495,496,332.0,108.0,5.0,4.5,4.0,9.02,1,0.87
496,497,337.0,117.0,5.0,5.0,5.0,9.87,1,0.96
497,498,330.0,120.0,5.0,4.5,5.0,9.56,1,0.93
498,499,312.0,103.0,4.0,4.0,5.0,8.43,0,0.73


In [9]:
df.describe()
## in case we see that our dataset having some sort of "Null_Value"

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
count,500.000000,485.000000,490.000000,485.000000,500.000000,500.00000,500.000000,500.000000,500.00000
mean,250.500000,316.558763,107.187755,3.121649,3.374000,3.48400,8.576440,0.560000,0.72174
std,144.481833,11.274704,6.112899,1.146160,0.991004,0.92545,0.604813,0.496884,0.14114
min,1.000000,290.000000,92.000000,1.000000,1.000000,1.00000,6.800000,0.000000,0.34000
25%,125.750000,308.000000,103.000000,2.000000,2.500000,3.00000,8.127500,0.000000,0.63000
50%,250.500000,317.000000,107.000000,3.000000,3.500000,3.50000,8.560000,1.000000,0.72000
75%,375.250000,325.000000,112.000000,4.000000,4.000000,4.00000,9.040000,1.000000,0.82000
max,500.000000,340.000000,120.000000,5.000000,5.000000,5.00000,9.920000,1.000000,0.97000


Fulfilled that null value shell. 
* this null value futher create the error during hyper parameter tunning

In [10]:
df['GRE Score'] = df['GRE Score'].fillna(df['GRE Score'].mean())
df['TOEFL Score'] = df['TOEFL Score'].fillna(df['TOEFL Score'].mean())
df['University Rating'] = df['University Rating'].fillna(df['University Rating'].mean())

In [11]:
x= df.drop(columns=['Serial No.' ,'Chance of Admit' ])
x

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
0,337.000000,118.0,4.0,4.5,4.5,9.65,1
1,324.000000,107.0,4.0,4.0,4.5,8.87,1
2,316.558763,104.0,3.0,3.0,3.5,8.00,1
3,322.000000,110.0,3.0,3.5,2.5,8.67,1
4,314.000000,103.0,2.0,2.0,3.0,8.21,0
...,...,...,...,...,...,...,...
495,332.000000,108.0,5.0,4.5,4.0,9.02,1
496,337.000000,117.0,5.0,5.0,5.0,9.87,1
497,330.000000,120.0,5.0,4.5,5.0,9.56,1
498,312.000000,103.0,4.0,4.0,5.0,8.43,0


In [12]:
x.describe()
# now all null value problem is resolved

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
count,500.000000,500.000000,500.000000,500.000000,500.00000,500.000000,500.000000
mean,316.558763,107.187755,3.121649,3.374000,3.48400,8.576440,0.560000
std,11.103952,6.051338,1.128802,0.991004,0.92545,0.604813,0.496884
min,290.000000,92.000000,1.000000,1.000000,1.00000,6.800000,0.000000
25%,309.000000,103.000000,2.000000,2.500000,3.00000,8.127500,0.000000
50%,316.558763,107.000000,3.000000,3.500000,3.50000,8.560000,1.000000
75%,324.000000,112.000000,4.000000,4.000000,4.00000,9.040000,1.000000
max,340.000000,120.000000,5.000000,5.000000,5.00000,9.920000,1.000000


In [13]:
df.isnull().sum()


Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

In [14]:
y = df['Chance of Admit' ]
y

0      0.92
1      0.76
2      0.72
3      0.80
4      0.65
       ... 
495    0.87
496    0.96
497    0.93
498    0.73
499    0.84
Name: Chance of Admit, Length: 500, dtype: float64

In [15]:
from sklearn.preprocessing import StandardScaler
std_sca = StandardScaler()
x   = std_sca.fit_transform(x)

In [16]:
x

array([[ 1.84274116e+00,  1.78854223e+00,  7.78905651e-01, ...,
         1.09894429e+00,  1.77680627e+00,  8.86405260e-01],
       [ 6.70814288e-01, -3.10581135e-02,  7.78905651e-01, ...,
         1.09894429e+00,  4.85859428e-01,  8.86405260e-01],
       [ 5.12433309e-15, -5.27312752e-01, -1.07876604e-01, ...,
         1.73062093e-02, -9.54042814e-01,  8.86405260e-01],
       ...,
       [ 1.21170361e+00,  2.11937866e+00,  1.66568791e+00, ...,
         1.63976333e+00,  1.62785086e+00,  8.86405260e-01],
       [-4.10964364e-01, -6.92730965e-01,  7.78905651e-01, ...,
         1.63976333e+00, -2.42366993e-01, -1.12815215e+00],
       [ 9.41258951e-01,  9.61451165e-01,  7.78905651e-01, ...,
         1.09894429e+00,  7.67219636e-01, -1.12815215e+00]])

In [22]:
from sklearn.metrics import mean_squared_error
def objective(trail , data = x , target = y ):
    train_x , test_x ,train_y , test_y = train_test_split(data , target , test_size = .10,random_state = 30  )
    param = {
        #'tree_method':'gpu_hist',                                             # for the gpu uses 
        'lambda':trail.suggest_loguniform('lambda' , 1e-4,10.0),              # best possible lambda value b/w this range
        'alpha' :trail.suggest_loguniform('alpha' , 1e-4 , 10.0),             # best possible alpha value b/w this range (1e-4 to 10)
        'colsample_bytree' :trail.suggest_categorical('colsample_bytree' , [.1,.2,.3,.4,.5,.6,.7,.8,.9,1]),
        'subsample' :trail.suggest_categorical('subsample' , [.1,.2,.3,.4,.5,.6,.7,.8,.9,1]),
        'learning_rate' : trail.suggest_categorical('learning_rate' , [.00001,.0003,.008,.02,.01,1,8]),
        'n_estimator' :30000,
        'max_depth' :trail.suggest_categorical('max_depth', [3,4,5,6,7,8,9,10,11,12]),
        'random_state' :trail.suggest_categorical('random_state' , [10,20,30 ,2000 , 3454,243123]),
        'min_child_weight' :trail.suggest_int('min_child_weight' ,1,200)
    }
    xgb_reg_model = xgb.XGBRegressor(**param)
    xgb_reg_model.fit(train_x,train_y,eval_set = [(test_x,test_y)] , verbose = True )
    pred_xgb = xgb_reg_model.predict(test_x)
    rmse = mean_squared_error(test_y,pred_xgb)
    return rmse

# {'lambda': 0.5646022891371127,
#  'alpha': 0.35848071012137483,
#  'colsample_bytree': 0.9,
#  'subsample': 0.2,
#  'learning_rate': 1,
#  'max_depth': 11,
#  'random_state': 30,
#  'min_child_weight': 30}

In [18]:
x

array([[ 1.84274116e+00,  1.78854223e+00,  7.78905651e-01, ...,
         1.09894429e+00,  1.77680627e+00,  8.86405260e-01],
       [ 6.70814288e-01, -3.10581135e-02,  7.78905651e-01, ...,
         1.09894429e+00,  4.85859428e-01,  8.86405260e-01],
       [ 5.12433309e-15, -5.27312752e-01, -1.07876604e-01, ...,
         1.73062093e-02, -9.54042814e-01,  8.86405260e-01],
       ...,
       [ 1.21170361e+00,  2.11937866e+00,  1.66568791e+00, ...,
         1.63976333e+00,  1.62785086e+00,  8.86405260e-01],
       [-4.10964364e-01, -6.92730965e-01,  7.78905651e-01, ...,
         1.63976333e+00, -2.42366993e-01, -1.12815215e+00],
       [ 9.41258951e-01,  9.61451165e-01,  7.78905651e-01, ...,
         1.09894429e+00,  7.67219636e-01, -1.12815215e+00]])

In [20]:
# !NVIDIA-SMI

/bin/bash: NVIDIA-SMI: command not found


!NVIDIA-SMI                             
*list down all the gpu available in the system

In [55]:
find_param = optuna.create_study(direction='minimize')
find_param.optimize(objective,n_trials = 5)
find_param.best_trial.params

# Note
# most of the time there is catch issue occurs so you must restart and 
# reimport all lib.

[I 2022-09-07 13:36:43,642] A new study created in memory with name: no-name-66aa2eaf-1d6a-4048-8aee-08dc878b57b9


[13:36:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:0.079537
[1]	validation_0-rmse:0.085039
[2]	validation_0-rmse:0.105868
[3]	validation_0-rmse:0.10093
[4]	validation_0-rmse:0.099226
[5]	validation_0-rmse:0.102097
[6]	validation_0-rmse:0.094388
[7]	validation_0-rmse:0.104409
[8]	validation_0-rmse:0.108317
[9]	validation_0-rmse:0.132817
[10]	validation_0-rmse:0.119273
[11]	validation_0-rmse:0.130811
[12]	validation_0-rmse:0.132529
[13]	validation_0-rmse:0.143886
[14]	validation_0-rmse:0.144213
[15]	validation_0-rmse:0.150554
[16]	validation_0-rmse:0.152652
[17]	validation_0-rmse:0.178942
[18]	validation_0-rmse:0.17596
[19]	validation_0-rmse:0.185689
[20]	validation_0-rmse:0.197418
[21]	validation_0-rmse:0.178981
[22]	validation_0-rmse:0.217035
[23]	validation_0-rmse:0.211302
[24]	validation_0-rmse:0.23058
[25]	validation_0-rmse:0.211508
[26]	validation_0-rmse:0.191156
[27]	validation_0-rmse

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.



[37]	validation_0-rmse:0.315328
[38]	validation_0-rmse:0.312946
[39]	validation_0-rmse:0.3202
[40]	validation_0-rmse:0.32853
[41]	validation_0-rmse:0.385642
[42]	validation_0-rmse:0.414928
[43]	validation_0-rmse:0.426016
[44]	validation_0-rmse:0.460793
[45]	validation_0-rmse:0.421382
[46]	validation_0-rmse:0.480243
[47]	validation_0-rmse:0.531901
[48]	validation_0-rmse:0.540295
[49]	validation_0-rmse:0.526636
[50]	validation_0-rmse:0.525757
[51]	validation_0-rmse:0.519581
[52]	validation_0-rmse:0.554523
[53]	validation_0-rmse:0.528177
[54]	validation_0-rmse:0.516596
[55]	validation_0-rmse:0.475858
[56]	validation_0-rmse:0.495964
[57]	validation_0-rmse:0.520869
[58]	validation_0-rmse:0.588139
[59]	validation_0-rmse:0.591522
[60]	validation_0-rmse:0.584722
[61]	validation_0-rmse:0.604462
[62]	validation_0-rmse:0.573434
[63]	validation_0-rmse:0.621797
[64]	validation_0-rmse:0.622666
[65]	validation_0-rmse:0.689369
[66]	validation_0-rmse:0.764254
[67]	validation_0-rmse:0.831851
[68]	valida

[I 2022-09-07 13:36:44,223] Trial 0 finished with value: 3.3742473971873035 and parameters: {'lambda': 0.006262669212781454, 'alpha': 0.015957972600341336, 'colsample_bytree': 0.7, 'subsample': 0.2, 'learning_rate': 1, 'max_depth': 6, 'random_state': 20, 'min_child_weight': 3}. Best is trial 0 with value: 3.3742473971873035.


...

[13:36:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:0.234987
[1]	validation_0-rmse:0.234986
[2]	validation_0-rmse:0.234984
[3]	validation_0-rmse:0.234982
[4]	validation_0-rmse:0.23498
[5]	validation_0-rmse:0.234978
[6]	validation_0-rmse:0.234976
[7]	validation_0-rmse:0.234975
[8]	validation_0-rmse:0.234973
[9]	validation_0-rmse:0.234971
[10]	validation_0-rmse:0.234969
[11]	validation_0-rmse:0.234967
[12]	validation_0-rmse:0.234965
[13]	validation_0-rmse:0.234964
[14]	validation_0-rmse:0.234962
[15]	validation_0-rmse:0.23496
[16]	validation_0-rmse:0.234958
[17]	validation_0-rmse:0.234956
[18]	validation_0-rmse:0.234954
[19]	validation_0-rmse:0.234952
[20]	validation_0-rmse:0.234951
[21]	validation_0-rmse:0.234949
[22]	validation_0-rmse:0.234947
[23]	validation_0-rmse:0.234945
[24]	validation_0-rmse:0.234943
[25]	validation_0-rmse:0.234941
[26]	validation_0-rmse:0.234939
[27]	validation_0-rms

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.



[45]	validation_0-rmse:0.234905
[46]	validation_0-rmse:0.234903
[47]	validation_0-rmse:0.234901
[48]	validation_0-rmse:0.234899
[49]	validation_0-rmse:0.234897
[50]	validation_0-rmse:0.234896
[51]	validation_0-rmse:0.234894
[52]	validation_0-rmse:0.234892
[53]	validation_0-rmse:0.23489
[54]	validation_0-rmse:0.234888
[55]	validation_0-rmse:0.234886
[56]	validation_0-rmse:0.234884
[57]	validation_0-rmse:0.234882
[58]	validation_0-rmse:0.23488
[59]	validation_0-rmse:0.234878
[60]	validation_0-rmse:0.234877
[61]	validation_0-rmse:0.234875
[62]	validation_0-rmse:0.234873
[63]	validation_0-rmse:0.234871
[64]	validation_0-rmse:0.234869
[65]	validation_0-rmse:0.234867
[66]	validation_0-rmse:0.234866
[67]	validation_0-rmse:0.234864
[68]	validation_0-rmse:0.234862
[69]	validation_0-rmse:0.23486
[70]	validation_0-rmse:0.234858
[71]	validation_0-rmse:0.234856
[72]	validation_0-rmse:0.234854
[73]	validation_0-rmse:0.234852
[74]	validation_0-rmse:0.23485
[75]	validation_0-rmse:0.234849
[76]	validat

[I 2022-09-07 13:36:46,086] Trial 4 finished with value: 0.0551331077172423 and parameters: {'lambda': 0.0007970848248501602, 'alpha': 1.470767637658116, 'colsample_bytree': 0.4, 'subsample': 0.4, 'learning_rate': 1e-05, 'max_depth': 12, 'random_state': 243123, 'min_child_weight': 137}. Best is trial 3 with value: 0.006247802862814514.


{'lambda': 0.4039272305281061,
 'alpha': 3.0677372501273883,
 'colsample_bytree': 0.3,
 'subsample': 0.6,
 'learning_rate': 1,
 'max_depth': 6,
 'random_state': 20,
 'min_child_weight': 106}

#### n_trials
* no. of the permutation and combination test cases apply
* we can also increase the n_trails [it increase the consumption of the gpu]

In [24]:
xgb.XGBRFRegressor()

XGBRFRegressor()

In [27]:
df.describe()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.00000,500.000000,500.000000,500.00000
mean,250.500000,316.558763,107.187755,3.121649,3.374000,3.48400,8.576440,0.560000,0.72174
std,144.481833,11.103952,6.051338,1.128802,0.991004,0.92545,0.604813,0.496884,0.14114
min,1.000000,290.000000,92.000000,1.000000,1.000000,1.00000,6.800000,0.000000,0.34000
25%,125.750000,309.000000,103.000000,2.000000,2.500000,3.00000,8.127500,0.000000,0.63000
50%,250.500000,316.558763,107.000000,3.000000,3.500000,3.50000,8.560000,1.000000,0.72000
75%,375.250000,324.000000,112.000000,4.000000,4.000000,4.00000,9.040000,1.000000,0.82000
max,500.000000,340.000000,120.000000,5.000000,5.000000,5.00000,9.920000,1.000000,0.97000


In [28]:
find_param.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_alpha,params_colsample_bytree,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_random_state,params_subsample,state
0,0,0.052290,2022-09-07 13:04:17.917256,2022-09-07 13:04:18.343505,0 days 00:00:00.426249,0.002827,0.7,7.821317,0.00030,10,144,2000,1.0,COMPLETE
1,1,0.052244,2022-09-07 13:04:18.346949,2022-09-07 13:04:18.819609,0 days 00:00:00.472660,0.006029,0.4,0.151788,0.00030,7,57,10,1.0,COMPLETE
2,2,0.055135,2022-09-07 13:04:18.826215,2022-09-07 13:04:19.301349,0 days 00:00:00.475134,0.000467,0.4,6.414591,0.00001,7,184,243123,0.1,COMPLETE
3,3,0.008780,2022-09-07 13:04:19.304610,2022-09-07 13:04:19.854362,0 days 00:00:00.549752,0.000709,0.8,0.003867,1.00000,12,26,243123,0.7,COMPLETE
4,4,0.055134,2022-09-07 13:04:19.859183,2022-09-07 13:04:20.262670,0 days 00:00:00.403487,0.422036,0.1,0.001578,0.00001,5,182,10,0.1,COMPLETE


In [29]:
optuna.visualization.plot_optimization_history(find_param)

In [30]:
optuna.visualization.plot_slice(find_param)

In [31]:
optuna.visualization.plot_contour(find_param , params = ['alpha' , 'lambda'])

In [35]:
best_param = {
 'lambda': 0.0038666953097313307,
 'alpha': 0.0007094688924691217,
 'colsample_bytree': 0.8,
 'subsample': 0.7,
 'learning_rate': 1,
 'max_depth': 12,
 'random_state': 243123,
 'min_child_weight': 26}



# {'lambda': 0.5646022891371127,
#  'alpha': 0.35848071012137483,
#  'colsample_bytree': 0.9,
#  'subsample': 0.2,
#  'learning_rate': 1,
#  'max_depth': 11,
#  'random_state': 30,
#  'min_child_weight': 30}

In [36]:
xgb_final_model = xgb.XGBRegressor(**best_param)
xgb_final_model.fit(x,y)

[13:06:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(alpha=0.0007094688924691217, colsample_bytree=0.8,
             lambda=0.0038666953097313307, learning_rate=1, max_depth=12,
             min_child_weight=26, random_state=243123, subsample=0.7)

In [37]:
xgb_final_model.score(x,y)

0.9494438424756979

In [38]:
#classification task 

In [39]:
import optuna
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [40]:
df = pd.read_csv('winequality_red.csv')

In [41]:
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [42]:
x = df.drop(columns='quality')
y = df.quality 

In [43]:
df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


In [44]:
train_x , test_x , train_y , test_y= train_test_split(x , y , test_size = .20 )

# try to create 2d matrix overHere [ it may not imp below we manage all the cases]

In [45]:
np.asarray(train_x)   
# asarray try to convert train and test  



array([[1.22e+01, 3.40e-01, 5.00e-01, ..., 3.12e+00, 1.18e+00, 9.20e+00],
       [6.70e+00, 7.50e-01, 1.00e-02, ..., 3.55e+00, 6.10e-01, 1.28e+01],
       [9.60e+00, 6.00e-01, 5.00e-01, ..., 3.50e+00, 5.70e-01, 9.70e+00],
       ...,
       [6.20e+00, 4.30e-01, 2.20e-01, ..., 3.52e+00, 6.00e-01, 9.50e+00],
       [7.40e+00, 7.00e-01, 0.00e+00, ..., 3.51e+00, 5.60e-01, 9.40e+00],
       [1.04e+01, 4.40e-01, 4.20e-01, ..., 3.38e+00, 8.60e-01, 9.90e+00]])

In [46]:
xgb_classification = xgb.XGBClassifier()
xgb_classification.fit(train_x , train_y,eval_set  = [(test_x, test_y)])

[0]	validation_0-merror:0.43125
[1]	validation_0-merror:0.39375
[2]	validation_0-merror:0.403125
[3]	validation_0-merror:0.396875
[4]	validation_0-merror:0.40625
[5]	validation_0-merror:0.39375
[6]	validation_0-merror:0.40625
[7]	validation_0-merror:0.390625
[8]	validation_0-merror:0.39375
[9]	validation_0-merror:0.39375
[10]	validation_0-merror:0.39375
[11]	validation_0-merror:0.396875
[12]	validation_0-merror:0.39375
[13]	validation_0-merror:0.3875
[14]	validation_0-merror:0.3875
[15]	validation_0-merror:0.378125
[16]	validation_0-merror:0.378125
[17]	validation_0-merror:0.38125
[18]	validation_0-merror:0.38125
[19]	validation_0-merror:0.375
[20]	validation_0-merror:0.38125
[21]	validation_0-merror:0.378125
[22]	validation_0-merror:0.38125
[23]	validation_0-merror:0.3875
[24]	validation_0-merror:0.384375
[25]	validation_0-merror:0.39375
[26]	validation_0-merror:0.39375
[27]	validation_0-merror:0.39375
[28]	validation_0-merror:0.403125
[29]	validation_0-merror:0.4
[30]	validation_0-me

XGBClassifier(objective='multi:softprob')

In [49]:
def objective_classification(trial) :
    train_x , test_x , train_y , test_y= train_test_split(x , y , test_size = .20 )
    
    param  = {
        #'tree_method' :'gpu_hist',         # this occurs the cuda error ("GPU"), so we can implement without it so it run on the system
        'verbosity' : 3,
        'objective' : "binary:logistics" , 
        'booster' : trial.suggest_categorical('booster' , ['dart' , 'gbtree','gblinear']),
        'lambda' : trial.suggest_float('lambda' , 1e-4 , 1),
        'alpha' :trial.suggest_float('alpha' , 1e-4 , 1),
        'subsample' : trial.suggest_float('subsample' , .1,.5),
        'colsample_bytree' : trial.suggest_float('colsample_bytree' , .1 ,.5)
        
    }
   
    if param['booster'] in ['gbtree' , 'dart']:
        param['gamma'] :trial.suggest_float('gamma' , 1e-3 , 4 )
        param['eta'] : trial.suggest_float('eta' , .001 ,5 )
        
    xgb_classification = xgb.XGBClassifier(**param)
    xgb_classification.fit(train_x , train_y,eval_set  = [(test_x, test_y)])
    pred = xgb_classification.predict(test_x)
    accuracy  = xgb_classification.score(test_x , test_y)

    
    
    return accuracy
        
        
    

In [50]:
xgb_classification_oputuna = optuna.create_study(direction  = 'minimize')
xgb_classification_oputuna.optimize(objective_classification , n_trials = 10 )

[I 2022-09-07 13:10:30,626] A new study created in memory with name: no-name-b1b17574-cd85-40cc-9a4b-0a34ddf9ae21


[13:10:30] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8 extra nodes, 0 pruned nodes, max_depth=3
[13:10:30] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[13:10:30] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[13:10:30] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[13:10:30] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[13:10:30] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 0 pruned nodes, max_depth=3
[13:10:30] INFO: /workspace/src/gbm/gbtree.cc:479: drop 0 trees, weight = 1
[0]	validation_0-merror:0.45625
[13:10:30] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max

[I 2022-09-07 13:10:33,935] Trial 0 finished with value: 0.6375 and parameters: {'booster': 'dart', 'lambda': 0.6520733905149579, 'alpha': 0.853343425982468, 'subsample': 0.42640736361605125, 'colsample_bytree': 0.40833602280161463}. Best is trial 0 with value: 0.6375.


[13:10:33] ======== Monitor: Learner ========
[13:10:33] EvalOneIter: 0.527038s, 100 calls @ 5270us
[13:10:33] GetGradient: 0.0381829s, 100 calls @ 381us
[13:10:33] PredictRaw: 1.77003s, 100 calls @ 17700us
[13:10:33] UpdateOneIter: 2.42659s, 100 calls @ 24265us
[13:10:33] ======== Monitor: GBTree ========
[13:10:33] BoostNewTrees: 0.55285s, 100 calls @ 5528us
[13:10:33] CommitModel: 0.0646188s, 100 calls @ 646us
[0]	validation_0-merror:0.546875
[1]	validation_0-merror:0.54375
[2]	validation_0-merror:0.528125
[3]	validation_0-merror:0.534375
[4]	validation_0-merror:0.54375
[5]	validation_0-merror:0.546875
[6]	validation_0-merror:0.54375
[7]	validation_0-merror:0.51875
[8]	validation_0-merror:0.521875
[9]	validation_0-merror:0.53125
[10]	validation_0-merror:0.5375
[11]	validation_0-merror:0.534375
[12]	validation_0-merror:0.534375
[13]	validation_0-merror:0.534375
[14]	validation_0-merror:0.534375
[15]	validation_0-merror:0.534375
[16]	validation_0-merror:0.5375
[17]	validation_0-merror

[I 2022-09-07 13:10:34,657] Trial 1 finished with value: 0.50625 and parameters: {'booster': 'gblinear', 'lambda': 0.3795675232779022, 'alpha': 0.31774239313085606, 'subsample': 0.3976660066025185, 'colsample_bytree': 0.3360088295598212}. Best is trial 1 with value: 0.50625.


...


[13:10:45] ======== Monitor: GBTree ========
[13:10:45] BoostNewTrees: 0.411404s, 100 calls @ 4114us
[13:10:45] CommitModel: 0.133484s, 100 calls @ 1334us
[13:10:45] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[13:10:45] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=3
[13:10:45] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[13:10:45] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[13:10:45] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[13:10:45] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8 extra nodes, 0 pruned nodes, max_depth=3
[13:10:45] INFO: /workspace/src/gbm/gbtree.cc:479: drop 0 trees, weight =

[I 2022-09-07 13:10:49,476] Trial 8 finished with value: 0.60625 and parameters: {'booster': 'dart', 'lambda': 0.5243815500579697, 'alpha': 0.4815360391228269, 'subsample': 0.4840776290031513, 'colsample_bytree': 0.18091297656944147}. Best is trial 3 with value: 0.459375.


[13:10:49] ======== Monitor: Learner ========
[13:10:49] EvalOneIter: 0.660811s, 100 calls @ 6608us
[13:10:49] GetGradient: 0.0505449s, 100 calls @ 505us
[13:10:49] PredictRaw: 1.99811s, 100 calls @ 19981us
[13:10:49] UpdateOneIter: 2.69164s, 100 calls @ 26916us
[13:10:49] ======== Monitor: GBTree ========
[13:10:49] BoostNewTrees: 0.582247s, 100 calls @ 5822us
[13:10:49] CommitModel: 0.0598495s, 100 calls @ 598us
[0]	validation_0-merror:0.559375
[1]	validation_0-merror:0.559375
[2]	validation_0-merror:0.55625
[3]	validation_0-merror:0.55
[4]	validation_0-merror:0.5625
[5]	validation_0-merror:0.55
[6]	validation_0-merror:0.528125
[7]	validation_0-merror:0.5125
[8]	validation_0-merror:0.490625
[9]	validation_0-merror:0.490625
[10]	validation_0-merror:0.49375
[11]	validation_0-merror:0.5
[12]	validation_0-merror:0.5
[13]	validation_0-merror:0.496875
[14]	validation_0-merror:0.5
[15]	validation_0-merror:0.5
[16]	validation_0-merror:0.496875
[17]	validation_0-merror:0.49375
[18]	validation

[I 2022-09-07 13:10:50,246] Trial 9 finished with value: 0.521875 and parameters: {'booster': 'gblinear', 'lambda': 0.7676586517019521, 'alpha': 0.5445656466307325, 'subsample': 0.35451965268742025, 'colsample_bytree': 0.341175784900145}. Best is trial 3 with value: 0.459375.


[13:10:50] PredictBatchInternal: 0.185514s, 203 calls @ 913us


In [51]:
xgb_classification_oputuna.best_trial

FrozenTrial(number=3, values=[0.459375], datetime_start=datetime.datetime(2022, 9, 7, 13, 10, 35, 374198), datetime_complete=datetime.datetime(2022, 9, 7, 13, 10, 36, 114278), params={'booster': 'gblinear', 'lambda': 0.18705008258964528, 'alpha': 0.038768513578730776, 'subsample': 0.26941566218393764, 'colsample_bytree': 0.2963325235904771}, distributions={'booster': CategoricalDistribution(choices=('dart', 'gbtree', 'gblinear')), 'lambda': FloatDistribution(high=1.0, log=False, low=0.0001, step=None), 'alpha': FloatDistribution(high=1.0, log=False, low=0.0001, step=None), 'subsample': FloatDistribution(high=0.5, log=False, low=0.1, step=None), 'colsample_bytree': FloatDistribution(high=0.5, log=False, low=0.1, step=None)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=3, state=TrialState.COMPLETE, value=None)

In [52]:
param = xgb_classification_oputuna.best_trial.params

In [53]:
xgb_final_clas = xgb.XGBClassifier(**param)
xgb_final_clas.fit(train_x , train_y)

XGBClassifier(alpha=0.038768513578730776, booster='gblinear',
              colsample_bytree=0.2963325235904771, lambda=0.18705008258964528,
              objective='multi:softprob', subsample=0.26941566218393764)

In [54]:
xgb_final_clas.score(test_x , test_y)

0.459375

### Note:
* Score is very less because we not complete flexiblity of the parameter and in case it not try to test all posible value in case i resticted 

* * Some time during the model or parameter selection (oputuna) "Chache" occur which lead to error [at that point time -- restart the model import all lib and then run command]
* OR Restart the kernal it not syn with python complier someting inside our catche memeory